# 地理信息系统设计与开发（重投影shp文件）

In [11]:
import os
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def myCreatePolygonShp():
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    outputfn='yunnan.shp'
    if os.path.exists(outputfn):
        shpdriver.DeleteDataSource(outputfn)
    outputds = shpdriver.CreateDataSource(outputfn)
    targetSR = osr.SpatialReference()
    targetSR.ImportFromEPSG(4326) #Geo WGS84
    outlyr = outputds.CreateLayer(outputfn,targetSR,geom_type=ogr.wkbPolygon)
    featureDefn = outlyr.GetLayerDefn()
    ds=ogr.Open('yunnan.json')
    lyr=ds.GetLayer()
    for feature in lyr:
        geom=feature.geometry()
        #print(geom)
        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geom)
        #print("outFeature:",outFeature.geometry())
        outlyr.CreateFeature(outFeature)
        outFeature = None 

    ds=None    
    outputds = None   

## EPSG:4326-->WGS 84
## EPSG:32648-->WGS 84 / UTM zone 48N
## 不能对multipolygon直接投影，只能对polygon进行投影，
## multipolygon只能拆解为polygon后投影才会正确

## 输入的空间参考

In [12]:
# input SpatialReference
inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(4326)

0

## 输出的空间参考

In [13]:
# output SpatialReference
outSpatialRef = osr.SpatialReference()
outSpatialRef.ImportFromEPSG(32648)

0

## 创建坐标转换参数

In [14]:
# create the CoordinateTransformation
coordTrans = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
print("coordTrans:",coordTrans)

coordTrans: <osgeo.osr.CoordinateTransformation; proxy of <Swig Object of type 'OSRCoordinateTransformationShadow *' at 0x00000254C75F4D80> >


## 建立driver

In [15]:
driver = ogr.GetDriverByName('ESRI Shapefile')

## 获取输入数据及其空间参考

In [16]:
# get the input layer
filename='yunnan.shp'
inDataSet = driver.Open(filename)
inLayer = inDataSet.GetLayer()
inspatialRef = inLayer.GetSpatialRef()
print(inspatialRef.ExportToWkt())

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


## 创建输出数据并设定空间参考

In [17]:
# create the output layer
outputShapefile = 'yunnan_32648.shp'
if os.path.exists(outputShapefile):
    driver.DeleteDataSource(outputShapefile)
outDataSet = driver.CreateDataSource(outputShapefile)
outLayer = outDataSet.CreateLayer("yunnan_32648", outSpatialRef, geom_type=ogr.wkbMultiPolygon)

outspatialRef = outLayer.GetSpatialRef()

print(outspatialRef.ExportToWkt())

PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]


## 建立输入文件和输出文件间空间参考转换参数

In [18]:
# create the CoordinateTransformation
coordTrans = osr.CoordinateTransformation(inspatialRef, outspatialRef)

## 构建与输入数据相同的属性表结构

In [19]:
# add fields
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    outLayer.CreateField(fieldDefn)

# get the output layer's feature definition
outLayerDefn = outLayer.GetLayerDefn()

## 循环对输入数据的每一个Geometry进行坐标变换后写入输出数据

In [20]:
# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
    # get the input geometry
    geom = inFeature.GetGeometryRef()
    print("geom old:",geom)
    spatialRef = geom.GetSpatialReference()
    print(spatialRef)
    # reproject the geometry
    geom.Transform(coordTrans)
    print("geom new:",geom)
    spatialRef = geom.GetSpatialReference()
    print(spatialRef)

    # create a new feature
    outFeature = ogr.Feature(outLayerDefn)
    # set the geometry and attribute
    outFeature.SetGeometry(geom)

    # for i in range(0, outLayerDefn.GetFieldCount()):
    #     outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))

    # add the feature to the shapefile
    outLayer.CreateFeature(outFeature)

    # dereference the features and get the next input feature
    outFeature = None
    inFeature = inLayer.GetNextFeature()

geom old: POLYGON ((99.5142 23.0823,99.5032 23.1152,99.5911 23.1702,99.624 23.1537,99.624 23.1921,99.679 23.2471,99.7998 23.1976,99.8438 23.2086,99.8987 23.2526,100.0195 23.2141,100.0745 23.2306,100.0964 23.28,100.0745 23.3185,100.0854 23.3459,100.1074 23.3789,100.1074 23.4174,100.1514 23.4723,100.1843 23.6096,100.2393 23.6096,100.2832 23.6426,100.3162 23.6151,100.3491 23.6151,100.437 23.6646,100.3821 23.6975,100.3821 23.725,100.3491 23.7524,100.3601 23.7799,100.3491 23.8129,100.3601 23.8568,100.4041 23.8733,100.415 23.9172,100.3821 23.9941,100.3601 24.0546,100.4919 24.0875,100.5249 24.1974,100.5029 24.2578,100.5139 24.2688,100.5029 24.3347,100.5029 24.4281,100.5469 24.472,100.5249 24.5105,100.5029 24.5215,100.4919 24.5764,100.426 24.6478,100.4041 24.7083,100.481 24.7357,100.5359 24.7083,100.5688 24.7083,100.6238 24.7412,100.6348 24.8236,100.7446 24.7852,100.7996 24.7247,100.8215 24.7357,100.8765 24.7247,100.9094 24.7467,100.9094 24.7247,100.9424 24.7247,100.9534 24.6698,101.0413 24.55

## 转换完毕，删除dataset

In [21]:
inDataSet = None
outDataSet = None